In [1]:
import product_definitions

In [2]:
dir(product_definitions)

['CallableBondStaticBase',
 'ConvertibleBondStaticBase',
 'EuropeanOptionStatic',
 'MBSPoolStatic',
 'ProductStaticBase',
 'QuantLibBondStaticBase',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_parse_date_input',
 '_parse_date_list',
 '_serialize_date_list',
 'abc',
 'date',
 'datetime',
 'np',
 'ql',
 'relativedelta']

In [3]:
import os
os.cpu_count()

24